## INSERT THE RQ HERE

## **Introduction:**

INSERT INTRO HERE 

1. **Dataset Exploration**
   - The process begins with importing Reddit comments and Comment attributes from a CSV file. 

2. **Data Preprocessing and Cleaning**
   - We start by loading Reddit comments from a CSV file and employ rigorous data cleaning methods. This includes removing URLs, HTML tags, special characters, and employing lemmatization for text normalization.

3. **Sentiment Analysis:**
- The VADER sentiment analysis tool is employed to evaluate the emotional content of each comment. This approach calculates four main sentiment scores—Compound, Positive, Negative, and Neutral—to provide a detailed insight into user emotions.

4. **Bias Analysis:**



In [2]:
!pip install nltk 
!pip install wordcloud

import nltk
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import re
import numpy as np
from bs4 import BeautifulSoup
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import os
from urllib.parse import urlparse
import re
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')       # for word_tokenize
nltk.download('wordnet')     # for WordNetLemmatizer
nltk.download('stopwords')   # for stopwords
nltk.download('vader_lexicon') # for Vader


[nltk_data] Downloading package punkt to /Users/onurguzel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/onurguzel/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/onurguzel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/onurguzel/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [3]:
# Read post data
df = pd.read_csv("/Users/onurguzel/Desktop/Files/Vrije/The Social Web/Project/archive/legacy/pse_isr_reddit_comments.csv")
# Read post attributes
df1 = pd.read_csv("/Users/onurguzel/Desktop/Files/Vrije/The Social Web/Project/archive/reddit_opinion_PSE_ISR.csv")


In [4]:
#Display Sample Data and Schema
df.sample(10)

,comment_id,score,self_text,subreddit,created_time
57486,k9ab0tx,11,You wot m8. Officials were repeatedly warned b...,worldnews,2023-11-15 00:00:59+00:00
654133,k4ooj54,1,steve is the leader of Hamas,NonCredibleDefense,2023-10-13 09:48:30+00:00
737134,k42rk7g,32,AC-130s are only operated by USAF and I doubt ...,CombatFootage,2023-10-09 02:08:01+00:00
97687,k8wuzao,3,When did that certified xenophobic c*#k Womble...,PublicFreakout,2023-11-12 10:27:33+00:00
655626,k4oi1hy,2,They will be refugees and rely on the mercy of...,CombatFootage,2023-10-13 08:23:05+00:00
610864,k4wcytk,1,That is not at all true. The Syrian Jewish com...,IsraelPalestine,2023-10-14 21:52:26+00:00
304114,k77wd6o,4,literally 0u812\n\n\nWait never mind was liste...,NonCredibleDefense,2023-10-31 12:14:47+00:00
342081,k6xwkfp,2,Unfortunately it's very difficult to different...,IsraelPalestine,2023-10-29 11:54:58+00:00
420921,k6apnw4,6,Not all Syrian rebels were funded by the Gulf ...,AskMiddleEast,2023-10-24 20:12:28+00:00
224556,k7tg9t9,1,Sweet!,IsrealPalestineWar_23,2023-11-04 18:04:55+00:00


In [5]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 769651 entries, 0 to 769650
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   comment_id    769651 non-null  object
 1   score         769651 non-null  int64 
 2   self_text     769649 non-null  object
 3   subreddit     769651 non-null  object
 4   created_time  769651 non-null  object
dtypes: int64(1), object(4)
memory usage: 29.4+ MB


In [6]:
for i in range(10):
    print(df.self_text[i])

Israel is like the most ethnically diverse country in the Middle East what are you talking about 💀
The only people starving them is Hamas who keeps hoarding all the aid food and shooting civilians who try to take it.
The casualty numbers for the pakistan-afghanistan conflict is actually nuts.
that's like the most notable thing about the video lmaoo. Nice one
'You thinking what I'm thinking?"

"Aim for the bushes"
Yup, still operating. The IDF has delivered fuel and meals to the hospital while working with staff on evacuating patients to safer places for care. While I’d presume the war hasn’t made things easier for the hospital, it is clear Hamas was already diminishing its capacity and capabilities by using it as a base. In addition to hoarding scarce resources, Hamas repurposed space in the hospital for their needs; for example a large store of ammunition and grenades were found hidden behind an MRI machine, which prevents the machine from operating at all.
\&gt;denies it's a dick mea

# Dataset Exploration

In [7]:
import pandas as pd
import plotly.express as px


unique_post_ids_count = df1['post_id'].nunique()
unique_comment_ids_count = df1['comment_id'].nunique()

data = {
    'Category': ['Number of Posts', 'Number of Comments'],
    'Count': [unique_post_ids_count, unique_comment_ids_count]
}
data_df = pd.DataFrame(data)

fig = px.bar(data_df, 
             x='Category', 
             y='Count', 
             title='Number of Unique Posts and Comments',
             color='Count', 
             color_continuous_scale=px.colors.sequential.Viridis)

fig.update_layout(paper_bgcolor='darkgrey', plot_bgcolor='darkgrey')

fig.update_yaxes(title='Count')

fig.show()


In [8]:
import pandas as pd
import plotly.express as px


author_counts = df1['author_name'].value_counts()

top_10_authors = author_counts.head(10)

fig = px.bar(top_10_authors, 
             x=top_10_authors.index, 
             y=top_10_authors.values,
             labels={'x': 'Author', 'y': 'Number of Posts/Comments'},
             title='Top 10 Authors by the number of Posts/Comments ',
             color=top_10_authors.values,
             color_continuous_scale=px.colors.sequential.Viridis)

fig.update_layout(paper_bgcolor='darkgrey', plot_bgcolor='darkgrey')

fig.update_xaxes(title='Author')
fig.update_yaxes(title='Number of Appearances')

for author, count in zip(top_10_authors.index, top_10_authors.values):
    fig.add_annotation(x=author, y=count,
                       text=str(count),
                       showarrow=False,
                       yshift=10)

fig.show()


In [9]:
import pandas as pd
import plotly.express as px

author_counts = df1['author_name'].value_counts()
top_50_authors_total = author_counts.head(50).sum()

total_posts = author_counts.sum()

rest_authors_total = total_posts - top_50_authors_total

data = {
    'Category': ['Top 50 Authors', 'Other Authors'],
    'Posts': [top_50_authors_total, rest_authors_total]
}
data_df = pd.DataFrame(data)

viridis_colors = ['#440154', '#3b528b', '#21908C', '#5DC863', '#FDE725'] 

fig = px.pie(data_df, 
             names='Category', 
             values='Posts', 
             title='Proportion of Content by Top 50 Authors vs Other Authors',
             color_discrete_sequence=viridis_colors)

fig.show()


In [10]:
import pandas as pd
import plotly.express as px

subreddit_counts = df['subreddit'].value_counts()

subreddits = subreddit_counts.head(5)

fig = px.bar(subreddits, 
             x=subreddits.index, 
             y=subreddits.values,
             labels={'x': 'Subreddit', 'y': 'Counts'},
             title='Most Posted Subreddits',
             color=subreddits.values,
             color_continuous_scale=px.colors.sequential.Viridis)

fig.update_layout(paper_bgcolor='darkgrey', plot_bgcolor='darkgrey')

fig.update_xaxes(title='Subreddit')
fig.update_yaxes(title='Count')

fig.show()


# Data Preprocessing and Cleaning

In [5]:
#Using further dependencies, please unzip the share wordnet.zip file to a path you choose
#!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

unzip:  cannot find or open /usr/share/nltk_data/corpora/wordnet.zip, /usr/share/nltk_data/corpora/wordnet.zip.zip or /usr/share/nltk_data/corpora/wordnet.zip.ZIP.


In [11]:
import zipfile
import os

#Change the directory depending on file location 
zip_file_path = '/Users/onurguzel/nltk_data/corpora/wordnet.zip'
destination_directory = '/Users/onurguzel/nltk_data/corpora'

file_to_check = 'index.adv'
file_path_to_check = os.path.join(destination_directory, 'wordnet', file_to_check)

if not os.path.exists(file_path_to_check):
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(destination_directory)
    print(f"Successfully unzipped {zip_file_path} to {destination_directory}")
else:
    print(f"File {file_to_check} already exists. No need to unzip the archive.")


File index.adv already exists. No need to unzip the archive.


In [12]:
import pandas as pd
from bs4 import BeautifulSoup
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

# Initialize the lemmatizer and stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def clean_text(text):
    if pd.isnull(text):
        return ""

    # Removing the URLs from the text
    text = re.sub(r'http\S+', '', text)
    
    # Remove HTML tags from the text
    soup = BeautifulSoup(text, 'html.parser')
    text = soup.get_text()
    
    # Removing the special characters from the text
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text).lower()
    
    tokens = word_tokenize(text)
    
    # Removing stopwords from the text
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words]
    
    cleaned_text = ' '.join(tokens)
    
    return cleaned_text

# Clean the self_text column and create clean_text 
# This can take up to 5mins depending on your processing power!!!
df['clean_text'] = df['self_text'].apply(clean_text)


/var/folders/bs/bcmtwb2145v942vknzgx45b00000gn/T/ipykernel_788/327348417.py:20: MarkupResemblesLocatorWarning:

The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.



In [13]:
#Comparison of the self_text and clean_text columns
print(df[['self_text', 'clean_text']])


                                                self_text  \
0       Israel is like the most ethnically diverse cou...   
1       The only people starving them is Hamas who kee...   
2       The casualty numbers for the pakistan-afghanis...   
3       that's like the most notable thing about the v...   
4       'You thinking what I'm thinking?"\n\n"Aim for ...   
...                                                   ...   
769646                               US. This is bullshit   
769647  I am in the United States and it has the dotte...   
769648  In which country are you?\nSometimes maps adap...   
769649  You can't give up on something you only preten...   
769650  &gt; The head of Islamic Jihad denounced Arab ...   

                                               clean_text  
0       israel like ethnically diverse country middle ...  
1       people starving hamas keep hoarding aid food s...  
2       casualty number pakistanafghanistan conflict a...  
3           thats like nota

In [14]:
#Display current state of the data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 769651 entries, 0 to 769650
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   comment_id    769651 non-null  object
 1   score         769651 non-null  int64 
 2   self_text     769649 non-null  object
 3   subreddit     769651 non-null  object
 4   created_time  769651 non-null  object
 5   clean_text    769651 non-null  object
dtypes: int64(1), object(5)
memory usage: 35.2+ MB


# Sentiment Analysis

In [15]:

analyzer = SentimentIntensityAnalyzer()

scores = {
    "Compound": [],
    "Positive": [],
    "Negative": [],
    "Neutral": []
}

# Generate sentiment scores for Reddit comments
for comment in df['clean_text'].dropna():
    score = analyzer.polarity_scores(comment)
    scores["Compound"].append(score["compound"])
    scores["Positive"].append(score["pos"])
    scores["Negative"].append(score["neg"])
    scores["Neutral"].append(score["neu"])

sentiments_score = pd.DataFrame(scores)

# This can take up to 3mins depending on your processing power!!!
df = pd.concat([df, sentiments_score], axis=1)

In [16]:
#Display sample 
df.head()

,comment_id,score,self_text,subreddit,created_time,clean_text,Compound,Positive,Negative,Neutral
0,k9tk1p2,1,Israel is like the most ethnically diverse cou...,IsraelPalestine,2023-11-18 22:58:37+00:00,israel like ethnically diverse country middle ...,0.3612,0.263,0.000,0.737
1,k9tjwfv,1,The only people starving them is Hamas who kee...,IsraelPalestine,2023-11-18 22:57:30+00:00,people starving hamas keep hoarding aid food s...,-0.4215,0.000,0.219,0.781
2,k9tjpdl,1,The casualty numbers for the pakistan-afghanis...,CombatFootage,2023-11-18 22:55:57+00:00,casualty number pakistanafghanistan conflict a...,-0.6597,0.130,0.570,0.300
3,k9tjoep,1,that's like the most notable thing about the v...,NonCredibleDefense,2023-11-18 22:55:44+00:00,thats like notable thing video lmaoo nice one,0.6486,0.469,0.000,0.531
4,k9tjicm,2,"'You thinking what I'm thinking?""\n\n""Aim for ...",NonCredibleDefense,2023-11-18 22:54:26+00:00,thinking im thinking aim bush,0.0000,0.000,0.000,1.000


In [17]:
#  <= -0.5, classify  'Negative'
#  > -0.5 and < 0.5, classify  'Neutral'
# >= 0.5, classify  'Positive'

conditions = [
    (df['Compound'] <= -0.5),
    (df['Compound'] > -0.5) & (df['Compound'] < 0.5),
    (df['Compound'] > 0.5)
]

values = ['Negative', 'Neutral', 'Positive']

df['Category'] = np.select(conditions, values)

In [18]:
df.head()

,comment_id,score,self_text,subreddit,created_time,clean_text,Compound,Positive,Negative,Neutral,Category
0,k9tk1p2,1,Israel is like the most ethnically diverse cou...,IsraelPalestine,2023-11-18 22:58:37+00:00,israel like ethnically diverse country middle ...,0.3612,0.263,0.000,0.737,Neutral
1,k9tjwfv,1,The only people starving them is Hamas who kee...,IsraelPalestine,2023-11-18 22:57:30+00:00,people starving hamas keep hoarding aid food s...,-0.4215,0.000,0.219,0.781,Neutral
2,k9tjpdl,1,The casualty numbers for the pakistan-afghanis...,CombatFootage,2023-11-18 22:55:57+00:00,casualty number pakistanafghanistan conflict a...,-0.6597,0.130,0.570,0.300,Negative
3,k9tjoep,1,that's like the most notable thing about the v...,NonCredibleDefense,2023-11-18 22:55:44+00:00,thats like notable thing video lmaoo nice one,0.6486,0.469,0.000,0.531,Positive
4,k9tjicm,2,"'You thinking what I'm thinking?""\n\n""Aim for ...",NonCredibleDefense,2023-11-18 22:54:26+00:00,thinking im thinking aim bush,0.0000,0.000,0.000,1.000,Neutral


In [19]:
category_counts = df.groupby(['Category'])['Category'].count()

counts_df = pd.DataFrame(category_counts).rename(columns={"Category": "Counts"})

counts_df['Percentage'] = (counts_df['Counts'] / counts_df['Counts'].sum()) * 100

counts_df = counts_df.drop(index='0')

print(counts_df)

          Counts  Percentage
Category                    
Negative  218908   28.442502
Neutral   431531   56.068400
Positive  119182   15.485200


In [22]:
import plotly.express as px

fig = px.bar(counts_df, x=counts_df.index, y='Counts',
             labels={'x': 'Sentiment Category', 'Counts': 'Count'},
             title='Sentiment Distribution within the Dataset',
             color='Counts', 
             color_continuous_scale=px.colors.sequential.Viridis)

fig.update_layout(paper_bgcolor='darkgrey', plot_bgcolor='darkgrey')

fig.update_yaxes(title='Count')

fig.show()


In [20]:
import plotly.express as px

# Assuming counts_df has a column 'Counts' and the index represents 'Sentiment Category'
fig = px.pie(counts_df, 
             names=counts_df.index, 
             values='Counts', 
             title='Sentiment Distribution within the Dataset',
             color_discrete_sequence=px.colors.sequential.Viridis[::2]) # Select discrete colors from Viridis

# Update layout for the dark grey background
fig.update_layout(
    paper_bgcolor='darkgrey', 
    plot_bgcolor='darkgrey',
    legend_title_text='Sentiment Category'
)

fig.show()


In [21]:
import plotly.express as px

top_comments = df.sort_values('score', ascending=False).drop_duplicates(subset='clean_text')[['clean_text', 'score']].head(10)

fig = px.bar(top_comments, 
             x='score', 
             y='clean_text', 
             orientation='h', 
             labels={'clean_text': 'Comment', 'score': 'Upvote Count'},
             title='Comments with High Upvotes',
             color='score',
             color_continuous_scale=px.colors.sequential.Viridis)

fig.update_layout(paper_bgcolor='white', plot_bgcolor='white')
fig.update_xaxes(title='Upvote Count')
fig.update_yaxes(title='Comment')
fig.update_layout(showlegend=False)

fig.update_yaxes(autorange="reversed")

fig.show()


In [22]:
import plotly.express as px

top_50_low_score_comments = df.sort_values('score', ascending=True).drop_duplicates(subset='clean_text')[['clean_text', 'score']].head(10)

fig = px.bar(top_50_low_score_comments, 
             x='score', 
             y='clean_text', 
             orientation='h', 
             labels={'clean_text': 'Comment', 'score': 'Upvote Count'},
             title='Comments with Low Upvotes',
             color='score',
             color_continuous_scale=px.colors.sequential.Viridis)

fig.update_layout(paper_bgcolor='white', plot_bgcolor='white')
fig.update_xaxes(title='Upvote Count')
fig.update_yaxes(title='Comment')
fig.update_layout(showlegend=False)

fig.update_yaxes(autorange="reversed")

fig.update_layout(height=350)  

fig.show()


# Bias Analysis

In [23]:
subreddit_sentiment = df.groupby('subreddit')[['Positive', 'Negative']].mean()

subreddit_sentiment['Bias'] = subreddit_sentiment['Positive'] - subreddit_sentiment['Negative']

def determine_bias_direction(bias):
    if bias > 0.01:  
        return 'Positive'
    elif bias < -0.01: 
        return 'Negative'
    else:
        return 'Neutral'

subreddit_sentiment['Bias Direction'] = subreddit_sentiment['Bias'].apply(determine_bias_direction)

most_biased_subreddits = subreddit_sentiment.sort_values(by='Bias', ascending=False)

print(most_biased_subreddits)


                       Positive  Negative      Bias Bias Direction
subreddit                                                         
AbruptChaos            0.170260  0.123790  0.046470       Positive
CrazyFuckingVideos     0.172251  0.158424  0.013827       Positive
NonCredibleDefense     0.157338  0.147750  0.009588        Neutral
Palestine              0.175961  0.188190 -0.012228       Negative
AskMiddleEast          0.166160  0.178500 -0.012340       Negative
ActualPublicFreakouts  0.175143  0.193862 -0.018719       Negative
IsraelPalestine        0.167693  0.187364 -0.019671       Negative
CombatFootage          0.158665  0.179134 -0.020469       Negative
worldnews              0.161203  0.186219 -0.025016       Negative
worldnewsvideo         0.173629  0.198985 -0.025357       Negative
TerrifyingAsFuck       0.163725  0.195814 -0.032089       Negative
PublicFreakout         0.160341  0.200716 -0.040376       Negative
IsrealPalestineWar_23  0.170765  0.219775 -0.049011       Nega

In [27]:
import pandas as pd
import plotly.express as px


top_5_positive = most_biased_subreddits[most_biased_subreddits['Bias Direction'] == 'Positive'].head(5)

top_5_negative = most_biased_subreddits[most_biased_subreddits['Bias Direction'] == 'Negative'].head(5)

fig_positive = px.bar(top_5_positive.reset_index(), 
                      x='subreddit', 
                      y='Bias',
                      labels={'subreddit': 'Subreddit', 'Bias': 'Bias Score'},
                      title='Top 5 Positively Biased Subreddits',
                      color='Bias',
                      color_continuous_scale=px.colors.sequential.Viridis)

fig_positive.update_layout(paper_bgcolor='darkgrey', plot_bgcolor='darkgrey')

fig_negative = px.bar(top_5_negative.reset_index(), 
                      x='subreddit', 
                      y='Bias',
                      labels={'subreddit': 'Subreddit', 'Bias': 'Bias Score'},
                      title='Top 5 Negatively Biased Subreddits',
                      color='Bias',
                      color_continuous_scale=px.colors.sequential.Viridis)

fig_negative.update_layout(paper_bgcolor='darkgrey', plot_bgcolor='darkgrey')

fig_positive.show()
fig_negative.show()
